In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

**Задание 1**

1. Загрузить, посмотреть, определить количество строк и объединить 3 датасета: marketing_campaign.csv, users.csv и subscribers.csv.
2. Определить типы и статистики колонок.
3. Определить эффективность маркетинговых каналов по привлечению платящих игроков.
4. Определить количество игроков в каждой возрастной группе.
5. Определить самую раннюю дату подписки на сервис.
6. Определить портрет аудитории удержанных подписчиков (по возрасту и языку).
7. Проверить гипотезу о том, в каком варианте теста (control/personalization) больше конверсия (converted) и значимо ли это отличие статистически.

In [3]:
# 1.1
df_camp = pd.read_csv(r'data_for_lessons\marketing_task\marketing_campaign.csv')

In [8]:
df_camp.info()
df_camp.shape
# 10037 строк и 6 столбцов

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10037 entries, 0 to 10036
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   user_id             10037 non-null  object
 1   date_served         10021 non-null  object
 2   marketing_channel   10022 non-null  object
 3   variant             10037 non-null  object
 4   language_displayed  10037 non-null  object
 5   converted           10022 non-null  object
dtypes: object(6)
memory usage: 470.6+ KB


(10037, 6)

In [4]:
df_sub = pd.read_csv(r'data_for_lessons\marketing_task\subscribers.csv')

In [9]:
df_sub.info()
df_sub.shape
# 10037 строк и 5 столбцов

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10037 entries, 0 to 10036
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   user_id              10037 non-null  object
 1   subscribing_channel  1856 non-null   object
 2   date_subscribed      1856 non-null   object
 3   date_canceled        577 non-null    object
 4   is_retained          1856 non-null   object
dtypes: object(5)
memory usage: 392.2+ KB


(10037, 5)

In [10]:
df_users = pd.read_csv(r'data_for_lessons\marketing_task\users.csv')

In [11]:
df_users.info()
df_users.shape
# 10037 строк и 3 столбца

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10037 entries, 0 to 10036
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   user_id             10037 non-null  object
 1   age_group           10037 non-null  object
 2   language_preferred  10037 non-null  object
dtypes: object(3)
memory usage: 235.4+ KB


(10037, 3)

In [12]:
df_merged = df_camp.merge(df_sub.merge(df_users, how = 'inner', on = 'user_id'), 
            how = 'inner', 
            on = 'user_id')

In [18]:
# 1.2
df_merged.info()
# Ответ: типы всех колонок - object, количество ненулевых значений видно ниже

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10037 entries, 0 to 10036
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   user_id              10037 non-null  object
 1   date_served          10021 non-null  object
 2   marketing_channel    10022 non-null  object
 3   variant              10037 non-null  object
 4   language_displayed   10037 non-null  object
 5   converted            10022 non-null  object
 6   subscribing_channel  1856 non-null   object
 7   date_subscribed      1856 non-null   object
 8   date_canceled        577 non-null    object
 9   is_retained          1856 non-null   object
 10  age_group            10037 non-null  object
 11  language_preferred   10037 non-null  object
dtypes: object(12)
memory usage: 941.1+ KB


In [25]:
# 1.3
df_merged.groupby('marketing_channel').agg({'converted':'sum'})
# Ответ: видим, что самый эффективный канал - House Ads. Примерно наравне с ним каналы Facebook и Instagram

,converted
marketing_channel,
Email,193
Facebook,237
House Ads,298
Instagram,265
Push,83


In [27]:
# 1.4
df_merged.groupby('age_group').agg({'user_id':'count'})

,user_id
age_group,
0-18 years,1539
19-24 years,1682
24-30 years,1568
30-36 years,1355
36-45 years,1353
45-55 years,1353
55+ years,1187


In [33]:
# 1.5
pd.to_datetime(df_merged['date_subscribed']).min()
# Ответ: 1 января 2018 года

C:\Users\iegor\AppData\Local\Temp\ipykernel_11904\356636381.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(df_merged['date_subscribed']).min()


Timestamp('2018-01-01 00:00:00')

In [39]:
# 1.6
df_retained = df_merged.query("is_retained == True")
df_retained['age_group'].value_counts()
# По распределению возрастных групп видим, что преобладают более молодые люди 0 - 30 лет
# Среди них преобладает группа людей возрастом от 19 до 24 лет 

age_group
19-24 years    326
24-30 years    273
0-18 years     229
30-36 years    122
45-55 years    119
36-45 years    115
55+ years       95
Name: count, dtype: int64

In [41]:
df_retained['language_preferred'].value_counts()
# Аудитория в подавляющем большинстве англоговорящая

# Ответ: итого аудитория удержанных подписчиков представляет собой англоговорящих людей
# возрастом преимущественно от 0 до 30 лет

language_preferred
English    1179
German       44
Spanish      44
Arabic       12
Name: count, dtype: int64

In [43]:
# 1.7 
df_merged.groupby('variant').agg({'converted':'sum'})
# Ответ: конверсия больше в варианте теста personalization практически в два раза
# Статистически значимо

,converted
variant,
control,371
personalization,705


**Задание 2**

Цех может производить стулья и столы. На производство стула идет 5 единиц материала, на производство стола - 20 единиц (футов красного дерева). Стул требует 10 человеко-часов, стол - 15. Имеется 400 единиц материала и 450 человеко-часов. Прибыль при производстве стула - 45 долларов США, при производстве стола - 80 долларов США. Сколько надо сделать стульев и столов, чтобы получить максимальную прибыль?

In [44]:
from scipy.optimize import linprog

c = [-45, -80]  # прибыли для стула и стола
A = [[5, 20], [10, 15]]  # материалы и человеко-часы
b = [400, 450]  # общие ресурсы

res = linprog(c, A_ub = A, b_ub = b, method = 'highs')

print("Количество стульев:", round(res.x[0]))
print("Количество столов:", round(res.x[1]))

Количество стульев: 24
Количество столов: 14


**Задание 3**

Средний накопленный доход на пользователя с сайта (cumulative ARPU) составил: day1 0.4$, day3 0.6$, day7 0.8$, day14 1.10$ , day21 1.30$, day30 1.40$. Подберите аппроксимирующую кривую и сделайте прогноз, сколько в среднем принесет денег пользователь на 90й день. Обоснуйте выбор одной из аппроксимирующей кривой вида: y = ab^x или y = aln(x) + b.

In [46]:
from scipy.optimize import curve_fit

days = np.array([1, 3, 7, 14, 21, 30])
arpu = np.array([0.4, 0.6, 0.8, 1.10, 1.30, 1.40])
# Функция для аппроксимации: a*b^x
def func(x, a, b):
    return a * np.power(b, x)
# Подгонка кривой
params, params_covariance = curve_fit(func, days, arpu)

a = params[0]
b = params[1]
# Прогноз для 90-го дня
day_90 = 90
arpu_90 = func(day_90, a, b)
print("Прогноз среднего накопленного дохода на 90-й день:", round(arpu_90, 2), "$")

Прогноз среднего накопленного дохода на 90-й день: 9.87 $
